In [1]:
import elliptic
import hashlib
import time
import json
import ecdsa
import random

class genKeyPair:
    def __init__(self):
        self.privateKey = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        self.publicKey = self.privateKey.get_verifying_key()
#     def __str__(self):
#         return 'VK '+ str(self.privateKey) + ' PK ' + str(self.publicKey.to_string().hex())
        
class Transaction:
    def __init__(self, send, receive, amount):
        self.send = send;
        self.receive = receive;
        self.amount = amount;
        
    def sign(self):
        self.signature = self.send.privateKey.sign(str(self.calHash()).encode())
        
    def calHash(self):
        header = str(self.send) + str(self.receive) + str(self.amount)
        return hashlib.sha256(header.encode()).hexdigest()
    
    def isValid(self,vk):
        try:
            vk.verify(self.signature, str(self.calHash()).encode())
        except ecdsa.keys.BadSignatureError:
            return False
        return True
        
    def __str__(self):
        return 'From: '+ str(self.send) + ' To: '+ str(self.receive) + ' Amount: ' + str(self.amount)

class Block:
    def __init__(self,transactions, prevH,nonce):
        self.transactions = transactions;
        self.prevH = prevH;
        self.timestamp = time.time();
        self.nonce = nonce;
        self.hash = self.calHash();
  

    def calHash(self):
        header = str(self.prevH) + str(self.transactions) + str(self.nonce) + str(self.timestamp)
        return hashlib.sha256(header.encode()).hexdigest()
  

    def mine(self,difficulty):
        if len(self.transactions) == 1:
            self.validTx()

        while True:
            self.hash = self.calHash();
            if int(self.hash,16) <= difficulty:
                self.hash = self.calHash();
                break
            else:
                self.nonce += 1
                continue
          
    
        print("Mine End", self.hash);
        
    def validTx(self):
        for i in self.transactions:
            if i.send == '':
                continue
            if not i.isValid(i.send.publicKey):
                print('Invalid Transaction Found')
#         return True        
        
    def __str__(self):
        return 'Tx: '+ str(self.transactions) + '\nPrev: ' + str(self.prevH) + '\nPow: ' + str(self.hash) + '\nNonce: '+ str(self.nonce) 



class Chain :
    def __init__(self):
        self.chain = [self.genesis()];
        self.pending = [];
        self.reward = random.randint(0,100);
        self.difficulty = 0x07FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF;
  
    def genesis(self) :
        genesis = Block("Genesis Block", '' ,random.randint(0,100));
        genesis.prevH =  genesis.calHash()
        return genesis;
  
    def addBlock(self,block):
        block.prevH = self.chain[-1].hash
#         transaction.hash = transaction.calHash();
        block.mine(self.difficulty)
        self.chain.append(block);        
         
    def addTransaction(self,transaction):
        if not transaction.send and transaction.receive:
            raise Warning('Invalid Send or Receive')
    
        if not transaction.isValid(transaction.send.publicKey) :
            raise Warning('Invalid transaction')
        else:
            print('Valid transaction')
        self.pending.append(transaction);

    def minePending(self,address) :

        minerReward = Transaction('',address,self.reward);
        self.pending.append(minerReward);


        newBlock = Block(self.pending,self.chain[-1].hash,1);
        newBlock.mine(self.difficulty);


        self.chain.append(newBlock);
        self.pending = [];
  
    def doubleSpending(self, transaction):
        for i in self.chain:
            if transaction in i.transactions:
                return True
        return False  

#     def detect_and_react_to_fork(self, chain):
#         if len(chain) > len(self.chain) and self.validateChain(chain):
#             self.chain = chain    
    
    def validateChain(self) :
        if len(self.chain) == 1 :
            if self.chain[0].hash != self.chain[0].calHash() :
                return False;
            return True

        for i in range(1,len(self.chain)):
            validate = self.chain[i];                          
#             if not validate.validTx():
#                 raise Warning('Invalid transaction')
#                 return False
            
            if (validate.hash != validate.calHash()) :
                raise Warning("Data tampering");
                return False;
      

            prev = self.chain[i-1];
            if validate.prevH != prev.hash :
                raise Warning("Block link broken");
                return False;
            
#             else:
#                 validate.addTransaction(validate)   
    
        return True;
  
class Node():
    def __init__(self,chain,transactions):
        self.chain=chain
        self.transactions=transactions
        
    def run(self):
        for i in self.transcations:
            if self.doubleSpending(i):
                raise Warning("Detection of double-spending");
            else:
                self.chain.addTransaction(i)



chain = Chain()

for i in range(0,8):
    text = 'No.' + str(i+1)
    block = Block(text,'',1)
    chain.addBlock(block)
print('\n')

for i in chain.chain:
    print(i)
# chain.validateChain()
print('\n')

# transaction 
sender = genKeyPair()
privateKeySender = sender.privateKey
publicKeySender = sender.publicKey

receiver = genKeyPair()
privateKeyReceived = receiver.privateKey
publicKeyReceived = receiver.publicKey

t1 = Transaction(sender,receiver,30)
t1.sign() 
print(t1.isValid(publicKeySender))

# t1.amount = 100
chain.addTransaction(t1)

chain.minePending(publicKeyReceived)
chain.validateChain()


# chain = Chain()
# add
# block2 = Block('transfer2','')
# chain.addBlockTOChhain(block2)
# block3 = Block('transfer3','')
# chain.addBlockTOChhain(block3)
# print('\n')
# for i in chain.chain:
#     print(i)
# chain.validateChain()
# print('\n')


# data tampering check
# chain.chain[1].transactions='transfer3'
# for i in chain.chain:
#     print(i)
# chain.validateChain()
# print('\n')

# chain.chain[1].transactions='transfer3'
# chain.chain[1].hash=chain.chain[1].computeHash()
# for i in chain.chain:
#     print(i)
# chain.validateChain()
# print('\n')

# add to pending pool
# tx1 = Transaction('A','B',1)
# tx2 = Transaction('B','A',10)
# chain.addTransaction(tx1)
# chain.addTransaction(tx2)

# for i in chain.chain:
#     print('Block')
#     print(i)
# print('\n')

# chain.minePending('C')

# for i in chain.chain[1].transactions:
#     print(i)

# for i in chain.chain:
#     print('Block')
#     print(i)
# print('\n')
# for i in chain.pending:
#     print('transactionPool')
#     print(i)
# print('\n')



# sign
# chains = Chain()
# chains.chain[0].mine(0x07FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF)

# sender = genKeyPair()
# privateKeySender = sender.privateKey
# publicKeySender = sender.publicKey
# # print(publicKeySender.to_string().hex())  

# receiver = genKeyPair()
# privateKeyReceived = receiver.privateKey
# publicKeyReceived = receiver.publicKey

# t1 = Transaction(sender,receiver,30)
# t1.sign()  
# print(t1.isValid(publicKeySender))

# # t1.amount = 100
# chains.addTransaction(t1)

# chains.minePending(publicKeyReceived)
# chains.validateChain()

for i in chain.chain:
    print(i)
    

Mine End 04c88389afe73e7ba9e50a7efd222f9e3b7b4f155ebbf1a09d531db7bb77eb51
Mine End 005e2bd7c22be702efee406f02a8891afdc5b3a02ccf17c79829964f420603ae
Mine End 00088ad86f61e618f2c5070bb6962e05f6c3188c7ccf6933cb68d22a4c81c803
Mine End 01f7f5bfb99c7ebd840381957e469effef32e19b2113cfeb7ae5551ce23377e9
Mine End 066912d1a231fdb639e3404bd7d8159ee9632976614878c4e8aff87c1082245f
Mine End 0560a071e0c8f9b35916119ae9930b5f9e7f6bca365268f02d7f54b1da4097ef
Mine End 05b276d0baadd30b2ff598c1e8d418e98b80cf9991c44f61c3bdec3110e0527a
Mine End 01475ca037cbc968ce5113ff496501ec233cdb279c5caf19ce9030e8b3462461


Tx: Genesis Block
Prev: 38ae225138dcd34f5412be43423c29e7e4198af249cbd9470394479f4e43cddf
Pow: 38ae225138dcd34f5412be43423c29e7e4198af249cbd9470394479f4e43cddf
Nonce: 15
Tx: No.1
Prev: 38ae225138dcd34f5412be43423c29e7e4198af249cbd9470394479f4e43cddf
Pow: 04c88389afe73e7ba9e50a7efd222f9e3b7b4f155ebbf1a09d531db7bb77eb51
Nonce: 111
Tx: No.2
Prev: 04c88389afe73e7ba9e50a7efd222f9e3b7b4f155ebbf1a09d531db7bb77e